In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## ขั้นตอนที่ 1: กำหนดโมเดล Pydantic สำหรับผลลัพธ์ที่มีโครงสร้าง

โมเดลเหล่านี้กำหนด **schema** ที่ตัวแทนจะส่งคืน การใช้ `response_format` ร่วมกับ Pydantic ช่วยให้:
- ✅ การดึงข้อมูลที่ปลอดภัยต่อประเภท
- ✅ การตรวจสอบอัตโนมัติ
- ✅ ไม่มีข้อผิดพลาดในการแปลงข้อความอิสระ
- ✅ การกำหนดเส้นทางตามเงื่อนไขได้ง่ายโดยอิงจากฟิลด์


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## ขั้นตอนที่ 2: สร้างเครื่องมือจองโรงแรม

เครื่องมือนี้คือสิ่งที่ **availability_agent** จะเรียกใช้เพื่อตรวจสอบว่ามีห้องว่างหรือไม่ เราใช้ตัวตกแต่ง `@ai_function` เพื่อ:
- เปลี่ยนฟังก์ชัน Python ให้เป็นเครื่องมือที่ AI สามารถเรียกใช้ได้
- สร้าง JSON schema สำหรับ LLM โดยอัตโนมัติ
- จัดการการตรวจสอบพารามิเตอร์
- เปิดใช้งานการเรียกใช้โดยอัตโนมัติผ่านตัวแทน

สำหรับการสาธิตนี้:
- **Stockholm, Seattle, Tokyo, London, Amsterdam** → มีห้องว่าง ✅
- **เมืองอื่นๆ ทั้งหมด** → ไม่มีห้องว่าง ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## ขั้นตอนที่ 3: กำหนดฟังก์ชันเงื่อนไขสำหรับการกำหนดเส้นทาง

ฟังก์ชันเหล่านี้จะตรวจสอบการตอบกลับของตัวแทนและตัดสินใจว่าจะเลือกเส้นทางใดในกระบวนการทำงาน

**รูปแบบสำคัญ:**
1. ตรวจสอบว่าข้อความเป็น `AgentExecutorResponse`
2. แยกวิเคราะห์ผลลัพธ์ที่มีโครงสร้าง (Pydantic model)
3. ส่งคืน `True` หรือ `False` เพื่อควบคุมการกำหนดเส้นทาง

กระบวนการทำงานจะประเมินเงื่อนไขเหล่านี้บน **edges** เพื่อเลือกตัวดำเนินการที่จะเรียกใช้ถัดไป


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## ขั้นตอนที่ 4: สร้าง Custom Display Executor

Executor เป็นส่วนประกอบของ workflow ที่ทำหน้าที่แปลงข้อมูลหรือสร้างผลกระทบข้างเคียง เราใช้ `@executor` decorator เพื่อสร้าง executor แบบกำหนดเองที่จะแสดงผลลัพธ์สุดท้าย

**แนวคิดสำคัญ:**
- `@executor(id="...")` - ใช้สำหรับลงทะเบียนฟังก์ชันเป็น workflow executor
- `WorkflowContext[Never, str]` - การระบุชนิดข้อมูลสำหรับ input/output
- `ctx.yield_output(...)` - ใช้สำหรับส่งผลลัพธ์สุดท้ายของ workflow


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## ขั้นตอนที่ 5: โหลดตัวแปรสภาพแวดล้อม

ตั้งค่าคลายแอนต์ LLM ตัวอย่างนี้ใช้งานร่วมกับ:
- **GitHub Models** (ระดับฟรีพร้อม GitHub token)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## ขั้นตอนที่ 6: สร้าง AI Agents พร้อมผลลัพธ์ที่มีโครงสร้าง

เราจะสร้าง **เอเจนต์เฉพาะทางสามตัว** โดยแต่ละตัวจะถูกห่อหุ้มใน `AgentExecutor`:

1. **availability_agent** - ตรวจสอบความพร้อมของโรงแรมโดยใช้เครื่องมือ
2. **alternative_agent** - แนะนำเมืองทางเลือก (เมื่อไม่มีห้องว่าง)
3. **booking_agent** - กระตุ้นให้ทำการจอง (เมื่อมีห้องว่าง)

**คุณสมบัติสำคัญ:**
- `tools=[hotel_booking]` - ให้เครื่องมือสำหรับเอเจนต์
- `response_format=PydanticModel` - บังคับให้ผลลัพธ์เป็น JSON ที่มีโครงสร้าง
- `AgentExecutor(..., id="...")` - ห่อหุ้มเอเจนต์เพื่อใช้ในกระบวนการทำงาน


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## ขั้นตอนที่ 7: สร้าง Workflow พร้อมเส้นทางแบบมีเงื่อนไข

ตอนนี้เราจะใช้ `WorkflowBuilder` เพื่อสร้างกราฟที่มีการกำหนดเส้นทางแบบมีเงื่อนไข:

**โครงสร้าง Workflow:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**วิธีการสำคัญ:**
- `.set_start_executor(...)` - กำหนดจุดเริ่มต้น
- `.add_edge(from, to, condition=...)` - เพิ่มเส้นทางแบบมีเงื่อนไข
- `.build()` - สร้าง Workflow ให้สมบูรณ์


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## ขั้นตอนที่ 8: ทดสอบกรณีที่ 1 - เมืองที่ไม่มีห้องว่าง (ปารีส)

ลองทดสอบเส้นทาง **ไม่มีห้องว่าง** โดยการขอข้อมูลโรงแรมในปารีส (ซึ่งไม่มีห้องว่างในสถานการณ์จำลองของเรา)


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## ขั้นตอนที่ 9: ทดสอบกรณีที่ 2 - เมืองที่มีห้องว่าง (สตอกโฮล์ม)

ตอนนี้เรามาทดสอบเส้นทาง **ห้องว่าง** โดยการขอข้อมูลโรงแรมในสตอกโฮล์ม (ซึ่งมีห้องว่างในสถานการณ์จำลองของเรา)


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## ประเด็นสำคัญและขั้นตอนถัดไป

### ✅ สิ่งที่คุณได้เรียนรู้:

1. **รูปแบบ WorkflowBuilder**
   - ใช้ `.set_start_executor()` เพื่อกำหนดจุดเริ่มต้น
   - ใช้ `.add_edge(from, to, condition=...)` สำหรับการกำหนดเส้นทางตามเงื่อนไข
   - เรียกใช้ `.build()` เพื่อสรุปการทำงานของ workflow

2. **การกำหนดเส้นทางตามเงื่อนไข**
   - ฟังก์ชันเงื่อนไขจะตรวจสอบ `AgentExecutorResponse`
   - แยกวิเคราะห์ผลลัพธ์ที่มีโครงสร้างเพื่อใช้ในการตัดสินใจเส้นทาง
   - คืนค่า `True` เพื่อเปิดใช้งานเส้นทาง หรือ `False` เพื่อข้าม

3. **การผสานรวมเครื่องมือ**
   - ใช้ `@ai_function` เพื่อแปลงฟังก์ชัน Python ให้เป็นเครื่องมือ AI
   - Agent จะเรียกใช้เครื่องมือโดยอัตโนมัติเมื่อจำเป็น
   - เครื่องมือจะคืนค่า JSON ที่ Agent สามารถแยกวิเคราะห์ได้

4. **ผลลัพธ์ที่มีโครงสร้าง**
   - ใช้โมเดล Pydantic สำหรับการดึงข้อมูลที่ปลอดภัยตามประเภท
   - ตั้งค่า `response_format=MyModel` เมื่อสร้าง Agent
   - แยกวิเคราะห์ผลลัพธ์ด้วย `Model.model_validate_json()`

5. **ตัวดำเนินการที่กำหนดเอง**
   - ใช้ `@executor(id="...")` เพื่อสร้างส่วนประกอบของ workflow
   - ตัวดำเนินการสามารถแปลงข้อมูลหรือทำผลกระทบข้างเคียง
   - ใช้ `ctx.yield_output()` เพื่อสร้างผลลัพธ์ของ workflow

### 🚀 การประยุกต์ใช้ในโลกจริง:

- **การจองการเดินทาง**: ตรวจสอบความพร้อม, แนะนำตัวเลือกอื่น, เปรียบเทียบตัวเลือก
- **บริการลูกค้า**: กำหนดเส้นทางตามประเภทปัญหา, อารมณ์, ลำดับความสำคัญ
- **อีคอมเมิร์ซ**: ตรวจสอบสต็อกสินค้า, แนะนำตัวเลือกอื่น, ดำเนินการสั่งซื้อ
- **การตรวจสอบเนื้อหา**: กำหนดเส้นทางตามคะแนนความไม่เหมาะสม, การแจ้งเตือนของผู้ใช้
- **กระบวนการอนุมัติ**: กำหนดเส้นทางตามจำนวนเงิน, บทบาทผู้ใช้, ระดับความเสี่ยง
- **การประมวลผลหลายขั้นตอน**: กำหนดเส้นทางตามคุณภาพและความสมบูรณ์ของข้อมูล

### 📚 ขั้นตอนถัดไป:

- เพิ่มเงื่อนไขที่ซับซ้อนมากขึ้น (หลายเกณฑ์)
- ใช้ลูปพร้อมการจัดการสถานะของ workflow
- เพิ่ม sub-workflows สำหรับส่วนประกอบที่นำกลับมาใช้ใหม่ได้
- ผสานรวมกับ API จริง (การจองโรงแรม, ระบบสต็อกสินค้า)
- เพิ่มการจัดการข้อผิดพลาดและเส้นทางสำรอง
- แสดงภาพ workflow ด้วยเครื่องมือแสดงภาพที่มีอยู่ในตัว



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามนุษย์ที่เป็นมืออาชีพ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดที่เกิดจากการใช้การแปลนี้
